In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
raw_2019 = pd.read_csv("./survey_results_public_2019.csv")

In [3]:
raw_2019.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [107]:
raw_2019['DevType'].head(20)

0                                                   NaN
1     Developer, desktop or enterprise applications;...
2     Designer;Developer, back-end;Developer, front-...
3                                 Developer, full-stack
4     Academic researcher;Developer, desktop or ente...
5     Data or business analyst;Data scientist or mac...
6                         Designer;Developer, front-end
7        Developer, back-end;Engineer, site reliability
8     Database administrator;Developer, back-end;Dev...
9     Data or business analyst;Data scientist or mac...
10                                                  NaN
11    Data or business analyst;Data scientist or mac...
12    Data or business analyst;Database administrato...
13        Developer, desktop or enterprise applications
14                                              Student
15                                Developer, full-stack
16    Developer, back-end;Developer, front-end;Devel...
17                             Data or business 

In [15]:
languages = raw_2019[['Respondent', 'LanguageWorkedWith']]

In [16]:
b = languages['LanguageWorkedWith'].str.split(';', expand=True)

In [29]:
l = []
for i in range(28):
    for j in b[i].unique():
        l.append(j)

In [32]:
all_lang = list(dict.fromkeys(l))

In [80]:
df_2019 = raw_2019[['Respondent', 'Employment', 'EdLevel', 'UndergradMajor', 'OrgSize','DevType', 'YearsCode', 'YearsCodePro', 'ConvertedComp', 'Age', 'Gender', 'Country']]

In [5]:
df_2019.head()

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,DevType,YearsCode,YearsCodePro,ConvertedComp,Age,Gender,Country
0,1,"Not employed, and not looking for work",Primary/elementary school,NaN,NaN,NaN,4,NaN,NaN,14.0,Man,United Kingdom
1,2,"Not employed, but looking for work","Secondary school (e.g. American high school, G...",NaN,NaN,"Developer, desktop or enterprise applications;...",NaN,NaN,NaN,19.0,Man,Bosnia and Herzegovina
2,3,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,1,8820.0,28.0,Man,Thailand
3,4,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,"Developer, full-stack",3,Less than 1 year,61000.0,22.0,Man,United States
4,5,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...","10,000 or more employees","Academic researcher;Developer, desktop or ente...",16,9,NaN,30.0,Man,Ukraine


In [6]:
df_2019.shape

(88883, 12)

In [81]:
df_2019 = df_2019[~df_2019['ConvertedComp'].isna()]

In [82]:
avg_by_country = df_2019.groupby(by=['Country'])['ConvertedComp'].mean()

In [83]:
df_2019 = df_2019.merge(avg_by_country, left_on='Country', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'CountryAvgComp'})

In [84]:
avg_by_ed_lvl = df_2019.groupby(by=['EdLevel'])['ConvertedComp'].mean()

In [85]:
df_2019 = df_2019.merge(avg_by_ed_lvl, left_on='EdLevel', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'EducationAvgComp'})

In [86]:
avg_by_major = df_2019.groupby(by=['UndergradMajor'])['ConvertedComp'].mean()

In [87]:
df_2019 = df_2019.merge(avg_by_major, left_on='UndergradMajor', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'MajorAvgComp'})

In [88]:
df_2019.groupby(by=['Gender']).count()

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,DevType,YearsCode,YearsCodePro,ConvertedComp,Age,Country,CountryAvgComp,EducationAvgComp,MajorAvgComp
Gender,,,,,,,,,,,,,,
Man,50503,50387,49787,46367,49671,49872,50441,50377,50503,48735,50503,50503,49787,46367
"Man;Non-binary, genderqueer, or gender non-conforming",115,115,111,112,113,114,115,115,115,113,115,115,111,112
"Non-binary, genderqueer, or gender non-conforming",342,341,334,300,333,335,339,338,342,318,342,342,334,300
Woman,3762,3754,3701,3500,3660,3703,3759,3747,3762,3642,3762,3762,3701,3500
Woman;Man,40,40,39,35,40,39,40,40,40,35,40,40,39,35
"Woman;Man;Non-binary, genderqueer, or gender non-conforming",25,24,22,16,24,24,22,22,25,22,25,25,22,16
"Woman;Non-binary, genderqueer, or gender non-conforming",106,106,106,88,103,105,104,106,106,105,106,106,106,88


In [89]:
df_2019['Gender'] = df_2019['Gender'].fillna('Not Specified')

In [90]:
le = preprocessing.LabelEncoder()
le.fit(list(df_2019['Gender'].unique()))

LabelEncoder()

In [91]:
df_2019['GenderCode'] = le.transform(df_2019['Gender'])

In [92]:
df_2019.loc[df_2019['OrgSize']=='1,000 to 4,999 employees', 'NewOrgSize'] = 3000
df_2019.loc[df_2019['OrgSize']=='10 to 19 employees', 'NewOrgSize'] = 15
df_2019.loc[df_2019['OrgSize']=='10,000 or more employees', 'NewOrgSize'] = 10000
df_2019.loc[df_2019['OrgSize']=='100 to 499 employees', 'NewOrgSize'] = 300
df_2019.loc[df_2019['OrgSize']=='2-9 employees', 'NewOrgSize'] = 5
df_2019.loc[df_2019['OrgSize']=='20 to 99 employees', 'NewOrgSize'] = 60
df_2019.loc[df_2019['OrgSize']=='5,000 to 9,999 employees', 'NewOrgSize'] = 7500
df_2019.loc[df_2019['OrgSize']=='500 to 999 employees', 'NewOrgSize'] = 750
df_2019.loc[df_2019['OrgSize']=='Just me - I am a freelancer, sole proprietor, etc.', 'NewOrgSize'] = 1

In [93]:
orgSizeMean = df_2019['NewOrgSize'].mean()

In [94]:
df_2019['NewOrgSize'] = df_2019['NewOrgSize'].fillna(value=orgSizeMean)

In [95]:
avgComp = df_2019['ConvertedComp'].mean()

In [96]:
df_2019['EducationAvgComp'] = df_2019['EducationAvgComp'].fillna(value=avgComp)

In [97]:
df_2019['MajorAvgComp'] = df_2019['MajorAvgComp'].fillna(value=avgComp)

In [98]:
df_2019 = df_2019.reset_index().drop('index', axis=1)

In [99]:
temp1 = df_2019['DevType'].str.split(';', expand=True)

In [39]:
temp1.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Developer, full-stack",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Database administrator,"Developer, back-end","Developer, front-end","Developer, full-stack","Developer, QA or test",DevOps specialist,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, front-end","Developer, full-stack","Developer, game or graphics",Educator,None,...,None,None,None,None,None,None,None,None,None,None


In [28]:
temp1.shape

(55823, 24)

In [100]:
all_positions = []
for row in range(temp1.shape[0]):
    j = temp1.iloc[row].as_matrix()
    for position in j:
        if position not in all_positions:
            all_positions.append(position)

/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
all_positions.remove(None)

In [102]:
all_positions.pop(-2)

nan

In [103]:
all_positions.sort()

In [104]:
all_positions

['Academic researcher',
 'Data or business analyst',
 'Data scientist or machine learning specialist',
 'Database administrator',
 'Designer',
 'DevOps specialist',
 'Developer, QA or test',
 'Developer, back-end',
 'Developer, desktop or enterprise applications',
 'Developer, embedded applications or devices',
 'Developer, front-end',
 'Developer, full-stack',
 'Developer, game or graphics',
 'Developer, mobile',
 'Educator',
 'Engineer, data',
 'Engineer, site reliability',
 'Engineering manager',
 'Marketing or sales professional',
 'Product manager',
 'Scientist',
 'Senior executive/VP',
 'Student',
 'System administrator']

In [105]:
for i in all_positions:
    df_2019[i] = 0

In [50]:
len(all_positions)

24

In [115]:
temp1[temp1]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Developer, full-stack",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Database administrator,"Developer, back-end","Developer, front-end","Developer, full-stack","Developer, QA or test",DevOps specialist,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, front-end","Developer, full-stack","Developer, game or graphics",Educator,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55818,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
55819,"Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, game or graphics","Developer, QA or test",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
55820,"Developer, full-stack","Engineer, site reliability",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
55821,Academic researcher,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, front-end","Developer, full-stack","Developer, game or graphics","Developer, mobile",Educator,...,None,None,None,None,None,None,None,None,None,None


In [118]:
for i in range(temp1.shape[0]):
    j = list(temp1.iloc[i].as_matrix())
    for item in j:
        if type(item)=='str':
            df_2019.at[i, item] = 1

/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [121]:
df_2019[all_positions].head(20)

,Academic researcher,Data or business analyst,Data scientist or machine learning specialist,Database administrator,Designer,DevOps specialist,"Developer, QA or test","Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices",...,Educator,"Engineer, data","Engineer, site reliability",Engineering manager,Marketing or sales professional,Product manager,Scientist,Senior executive/VP,Student,System administrator
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
5,0,1,0,1,0,1,0,1,1,0,...,0,1,1,1,0,0,0,1,0,1
6,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
df_2019 = df_2019.drop(['DevType', 'YearsCodePro'], axis=1)


In [123]:
df_2019.columns

Index(['Respondent', 'Employment', 'EdLevel', 'UndergradMajor', 'OrgSize',
       'YearsCode', 'ConvertedComp', 'Age', 'Gender', 'Country',
       'CountryAvgComp', 'EducationAvgComp', 'MajorAvgComp', 'GenderCode',
       'NewOrgSize', 'Academic researcher', 'Data or business analyst',
       'Data scientist or machine learning specialist',
       'Database administrator', 'Designer', 'DevOps specialist',
       'Developer, QA or test', 'Developer, back-end',
       'Developer, desktop or enterprise applications',
       'Developer, embedded applications or devices', 'Developer, front-end',
       'Developer, full-stack', 'Developer, game or graphics',
       'Developer, mobile', 'Educator', 'Engineer, data',
       'Engineer, site reliability', 'Engineering manager',
       'Marketing or sales professional', 'Product manager', 'Scientist',
       'Senior executive/VP', 'Student', 'System administrator'],
      dtype='object')

In [74]:
raw_2019.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [124]:
df_2019 = df_2019.merge(raw_2019[['Respondent', 'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'WebFrameWorkedWith']], on='Respondent', how='left')

In [125]:
temp2 = df_2019['LanguageWorkedWith'].str.split(';', expand=True)

In [126]:
all_languages = []
for row in range(temp2.shape[0]):
    j = temp2.iloc[row].as_matrix()
    for language in j:
        if language not in all_languages:
            all_languages.append(language)

/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [127]:
all_languages.remove(None)

In [128]:
all_languages.pop(-10)

nan

In [129]:
all_languages.sort()

In [130]:
for i in all_languages:
    df_2019[i] = 0

In [131]:
temp2.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,HTML/CSS,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,C,C++,C#,Python,SQL,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Java,R,SQL,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Bash/Shell/PowerShell,C#,HTML/CSS,JavaScript,Python,Ruby,Rust,SQL,TypeScript,WebAssembly,...,None,None,None,None,None,None,None,None,None,None
4,C#,Go,JavaScript,Python,R,SQL,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [132]:
for i in range(temp2.shape[0]):
    j = list(temp2.iloc[i].as_matrix())
    for item in j:
        if type(item)=='str':
            df_2019.at[i, item] = 1

/Users/owenbchiu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [133]:
df_2019[~df_2019['HTML/CSS'].isna()]

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,YearsCode,ConvertedComp,Age,Gender,Country,...,Python,R,Ruby,Rust,SQL,Scala,Swift,TypeScript,VBA,WebAssembly
0,3,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,100 to 499 employees,3,8820.0,28.0,Man,Thailand,...,0,0,0,0,0,0,0,0,0,0
1,4,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,3,61000.0,22.0,Man,United States,...,0,0,0,0,0,0,0,0,0,0
2,6,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,NaN,13,366420.0,28.0,Man,Canada,...,0,0,0,0,0,0,0,0,0,0
3,9,Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",10 to 19 employees,12,95179.0,23.0,Man,New Zealand,...,0,0,0,0,0,0,0,0,0,0
4,10,Employed full-time,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",NaN,"10,000 or more employees",12,13293.0,NaN,Not Specified,India,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55818,88878,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",20 to 99 employees,12,130000.0,26.0,Man,United States,...,0,0,0,0,0,0,0,0,0,0
55819,88879,Employed full-time,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",20 to 99 employees,17,82488.0,34.0,Man,Finland,...,0,0,0,0,0,0,0,0,0,0
55820,88881,Employed full-time,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,18,68745.0,37.0,Man,Austria,...,0,0,0,0,0,0,0,0,0,0
55821,88882,Employed full-time,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,10,588012.0,NaN,Man,Netherlands,...,0,0,0,0,0,0,0,0,0,0
